## IMPUTE INSANITY!:
### before we move onto neural networks we need to impute the NANs
#### How do we even know we've correctly imputed?
#### **Try out every impute methods and per feature set train the GBM and see which one got most importance**

### TODO
- #### [KNN , DeepLearner, other novel ways to impute](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779)
  - KNN thoughts: we can perhaps use subset of features with no nans, but then the records will become similar, and decision trees already predict the same result for similar looking rows? then what's the point of imputing value same as similar looking rows?
- ### DON't Impute rather provide the Flag wether the feature has value or not, kinda like mask; we did this in Optiver (not the transformer sequence mask , but we have mask about weather the second/grouped seconds is missing or not)
  - #### [Results on passing information about missing values to neural networks](https://machinelearningmastery.com/binary-flags-for-missing-values-for-machine-learning/)
- ### AMEND ABOVE: Impute as well as pass the missing Flag too, so neural network learns better
- ### Categorical Imputation: train on data with missing categorical values as target variable! **We can even use TEST set to do this!!!! yay!**
  - #### [Training to find missing categorical values](https://www.analyticsvidhya.com/blog/2021/04/how-to-handle-missing-values-of-categorical-variables/)

### IMPORTANT FLOAT16 aggregations return NONE!!! mean, sum most of it doesn't work :-/

## TODO:
- ### MULTILEVEL IMPUTATION DEFAULT
- ### LEVEL 1: out of all the variant+original; select the one with best amex scores DEFAULT_EMP1
    - #### This is partial nans
- ### Level 2: now that we have partial none df with best variants(including original with nans) ; TRAIN ML MODEL to have PREDICTED/BS variant
    - #### In the order of lowest missing values to highest missing values
    - #### we Target the feature with lowest amount on NAN as we have maximum training rows; then slowly cascade to more missing row features
    - #### once we have computed PREDICTED/BS variants for all of the features with nans
- ### Level 3: from DEFAULT_EMP1 vs PREDICTED variant select the one with maximum amex score (this is kinda like back to stage 1 but with improved train_df_wt)
    - #### repetetively select and choose the best varint out of the two
    - #### DEFAULT_EMP2 is generated!
    - #### Now there is a scope of repeating Level 2 again with using base_df of EMP2 and then going to EMP3,

In [1]:
WHERE_THIS = "local" # local|kaggle

BS_DATASET = "amex-default-prediction-binarysentient"
if WHERE_THIS == "kaggle":
    INPUT_PATH = "/kaggle/input/amex-default-prediction/"
    OUTPUT_PATH = "/kaggle/working/"
    TEMP_PATH = "/kaggle/temp/"
elif WHERE_THIS == "local":
    INPUT_PATH = "input/amex-default-prediction/"
    OUTPUT_PATH = "working/"
    TEMP_PATH = "temp/"


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import gc
import os
from datetime import datetime
import torch


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display, HTML
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.model_selection import StratifiedKFold
import plotly.express as pltex

import impute_insanity
from impute_insanity import load_prepare_amex_dataset
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk(INPUT_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

input/amex-default-prediction/sample_submission.csv
input/amex-default-prediction/test_data.csv
input/amex-default-prediction/train_data.csv
input/amex-default-prediction/train_labels.csv
input/amex-default-prediction/.ipynb_checkpoints\sample_submission-checkpoint.csv


In [3]:
object_features = ['customer_ID']
discard_features = ['D_66']
datetime_features = ['S_2']
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']
target_features = ['target']
numeric_features = ['R_2', 'S_25', 'D_118', 'B_5', 'D_60', 'B_21', 'D_115', 'S_15', 'D_84', 'D_122', 'B_9', 'B_33', 'D_53', 'R_24', 'D_94', 'D_56', 'D_139', 'R_28', 'B_3', 'S_20', 'B_31', 'D_133', 'B_2', 'S_12', 'D_71', 'D_96', 'S_7', 'D_72', 'B_36', 'B_41', 'S_5', 'D_41', 'R_22', 'R_8', 'D_140', 'D_47', 'D_89', 'P_2', 'R_19', 'D_59', 'B_23', 'S_3', 'D_145', 'D_103', 'B_19', 'R_20', 'D_73', 'D_136', 'D_141', 'D_142', 'B_22', 'D_46', 'B_29', 'B_25', 'D_128', 'B_18', 'D_86', 'D_109', 'B_8', 'B_17', 'R_17', 'B_12', 'D_54', 'D_74', 'S_16', 'B_6', 'D_49', 'D_80', 'S_8', 'B_7', 'D_144', 'B_27', 'B_26', 'R_25', 'R_23', 'D_82', 'D_111', 'B_10', 'D_113', 'R_4', 'D_48', 'R_26', 'S_23', 'B_11', 'D_104', 'D_134', 'D_79', 'P_3', 'D_132', 'D_137', 'D_135', 'B_28', 'D_88', 'S_13', 'D_51', 'D_61', 'D_75', 'D_69', 'R_16', 'S_6', 'S_17', 'D_93', 'B_20', 'D_112', 'D_123', 'D_130', 'B_1', 'D_78', 'D_92', 'S_27', 'D_44', 'B_16', 'R_5', 'D_43', 'S_18', 'B_15', 'D_39', 'D_50', 'D_55', 'S_9', 'D_105', 'D_70', 'R_18', 'D_125', 'D_58', 'S_24', 'D_110', 'D_42', 'R_6', 'D_81', 'R_7', 'D_138', 'D_52', 'R_27', 'D_124', 'D_45', 'D_91', 'D_108', 'S_22', 'B_14', 'D_83', 'R_13', 'D_87', 'S_19', 'D_131', 'R_21', 'B_40', 'R_3', 'D_65', 'B_13', 'D_129', 'D_119', 'B_32', 'R_9', 'B_24', 'D_127', 'D_106', 'D_102', 'R_1', 'R_14', 'B_37', 'D_107', 'R_15', 'R_12', 'P_4', 'R_10', 'customer_ID', 'D_121', 'R_11', 'S_11', 'D_76', 'D_143', 'B_39', 'B_42', 'D_62', 'B_4', 'S_26', 'D_77']
    

In [4]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def fast_amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

# Need Lightgbm supported eval metric
#Custom eval function expects a callable with following signatures: func(y_true, y_pred), func(y_true, y_pred, weight) or func(y_true, y_pred, weight, group) and returns (eval_name, eval_result, is_higher_better) or list of (eval_name, eval_result, is_higher_better):
def lgbm_eval_metric_amex(y_true, y_pred):
    amex_metric = fast_amex_metric(y_true, y_pred)
    return ('amex', amex_metric, True)

In [5]:
train_label_df = load_prepare_amex_dataset('train_labels')#, index_col='customer_ID')
train_label_df

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1.0


In [6]:
prediction_labels = train_label_df.copy()
# prediction_labels['prediction'] = prediction_labels['target'] 
# prediction_labels['prediction'] = prediction_labels['target'] - 3
prediction_labels['prediction'] = 1000
del prediction_labels['target'] 


### Predictions impact on Evaluation Metric
- #### all predictions 0 :   score 0.018829875288988086
- #### all predictions 1 :   score 0.018829875288988086
- #### all predictions 0.4 : score 0.018829875288988086
- #### same as target      : score 1.0
- #### target * 10         : score 1.0
- #### target * 0.001      : score 1.0
- #### target - 3          : score 1.0
- #### target + 3          : score 1.0

## The evaluation metrics is solid! amazing, need to understand more
## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays

In [7]:
## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays
# print(amex_metric(train_labels, prediction_labels))
print(fast_amex_metric(train_label_df['target'], prediction_labels['prediction']))

0.02260958442726859


## Precision impact on scores:

### float32: 0.788541
### float16: 0.788748, 0.788649, 0.788821
### float64: 0.788276, 0.789053, 0.789025, 0.788543


## Default

## EMP1


In [8]:
fold_score_track = {}
def track_score(key, score):
    key = str(key)
    if key not in fold_score_track:
        fold_score_track[key] = []
    fold_score_track[key].append(score)

def show_score(key):
    key = str(key)
    display(HTML(f"<h3>{key} OVERALL SCORE : {np.mean(fold_score_track[key]):0.6f}</h3>"))

In [9]:
def create_lgbm_model_with_config(random_state=1, n_estimators=2400, importance_type=None, early_stopping_rounds=None):
    """
    Creates model with our desired and some default hyper params
    importance_type: split|gain
    """
    if early_stopping_rounds is None:
        early_stopping_rounds = n_estimators//10
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=95, num_threads=12,
                          colsample_bytree=0.19,early_stopping_rounds=early_stopping_rounds,
                          max_bins=511, random_state=random_state, importance_type=importance_type)


In [10]:
%%time
features_avg = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_53', 'D_54', 'D_55', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_86', 'D_91', 'D_92', 'D_94', 'D_96', 'D_103', 'D_104', 'D_108', 'D_112', 'D_113', 'D_114', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_129', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_14', 'R_15', 'R_16', 'R_17', 'R_20', 'R_21', 'R_22', 'R_24', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_18', 'S_22', 'S_23', 'S_25', 'S_26']
features_avg = list(set(features_avg).difference(set(cat_features)))
features_min = ['B_2', 'B_4', 'B_5', 'B_9', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_19', 'B_20', 'B_28', 'B_29', 'B_33', 'B_36', 'B_42', 'D_39', 'D_41', 'D_42', 'D_45', 'D_46', 'D_48', 'D_50', 'D_51', 'D_53', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_70', 'D_71', 'D_74', 'D_75', 'D_78', 'D_83', 'D_102', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_128', 'D_132', 'D_140', 'D_141', 'D_144', 'D_145', 'P_2', 'P_3', 'R_1', 'R_27', 'S_3', 'S_5', 'S_7', 'S_9', 'S_11', 'S_12', 'S_23', 'S_25']
features_min = list(set(features_min).difference(set(cat_features)))
features_max = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_21', 'B_23', 'B_24', 'B_25', 'B_29', 'B_30', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_52', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_63', 'D_64', 'D_65', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_91', 'D_102', 'D_105', 'D_107', 'D_110', 'D_111', 'D_112', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_138', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_3', 'R_5', 'R_6', 'R_7', 'R_8', 'R_10', 'R_11', 'R_14', 'R_17', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']
features_max = list(set(features_max).difference(set(cat_features)))
features_last = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_86', 'D_91', 'D_96', 'D_105', 'D_106', 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124', 'D_125', 'D_126', 'D_127', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_138', 'D_140', 'D_141', 'D_142', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_19', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']
features_last = list(set(features_last).union(set(cat_features)))
for i in ['test', 'train']:
    df = load_prepare_amex_dataset(f"{i}_data_emp1")
    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer
    if 'target' in df.columns:
        df.drop(columns=['target'], inplace=True)
    gc.collect()
    print('Read', i)
    df_avg = (df
              .groupby(cid)
              .mean()[features_avg]
              .rename(columns={f: f"{f}_avg" for f in features_avg})
             )
    gc.collect()
    print('Computed avg', i)
    df_min = (df
              .groupby(cid)
              .min()[features_min]
              .rename(columns={f: f"{f}_min" for f in features_min})
             )
    gc.collect()
    print('Computed min', i)
    df_max = (df
              .groupby(cid)
              .max()[features_max]
              .rename(columns={f: f"{f}_max" for f in features_max})
             )
    gc.collect()
    print('Computed max', i)
    df = (df.loc[last, features_last]
          .rename(columns={f: f"{f}_last" for f in features_last})
          .set_index(np.asarray(cid[last]))
         )
    gc.collect()
    print('Computed last', i)
    df = pd.concat([df, df_min, df_max, df_avg], axis=1)
    if i == 'train': df_train = df
    else: df_test = df
    print(f"{i} shape: {df.shape}")
    del df, df_avg, df_min, df_max, cid, last

target = load_prepare_amex_dataset('train_labels').target.values
print(f"target shape: {target.shape}")

Read test
Computed avg test


<timed exec>:26: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.


Computed min test


<timed exec>:33: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.


Computed max test
Computed last test
test shape: (924621, 458)
Read train
Computed avg train


<timed exec>:26: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.


Computed min train


<timed exec>:33: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.


Computed max train
Computed last train
train shape: (458913, 458)
target shape: (458913,)
CPU times: total: 2min 53s
Wall time: 1min 55s


In [12]:
df_train_bs_features = load_prepare_amex_dataset("train_data_bs_features_extracted")
df_train = df_train.reset_index()
df_train = df_train.rename(columns={'index':'customer_ID'})
df_train = df_train.merge(df_train_bs_features, on="customer_ID", how="inner")

In [23]:
df_train

,customer_ID,D_54_last,B_16_last,D_49_last,S_20_last,R_5_last,B_2_last,R_14_last,D_138_last,S_22_last,...,B_10_avg,B_1_avg,D_118_avg,D_121_avg,D_125_avg,D_94_avg,B_14_avg,D_122_avg,R_22_avg,B_17_avg
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.008097,0.006408,NaN,0.009411,0.009444,1.007647,0.002902,0.164614,0.917811,...,0.270280,0.012007,0.245514,0.711829,0.006253,0.005909,0.023142,0.433732,0.005002,0.932087
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.002821,0.002940,NaN,0.009467,0.005880,1.004028,0.006271,0.164614,0.920889,...,0.298815,0.025654,0.433039,0.535892,0.004236,0.005067,0.014848,0.290804,0.003541,0.932087
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,1.005992,0.007836,NaN,0.009926,0.003000,0.812649,0.002310,0.164614,0.302868,...,0.273711,0.004386,0.354381,0.431903,0.004741,0.004365,0.004729,0.147314,0.005015,0.932087
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,1.004073,0.089177,NaN,0.007374,0.001347,1.006183,0.006133,0.164614,0.931634,...,0.306553,0.059876,0.048511,0.621386,0.004309,0.004629,0.033350,0.324807,0.005413,0.573924
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,1.007116,0.005095,NaN,0.002803,0.008859,0.815746,0.006752,0.164614,0.297843,...,0.100315,0.005941,0.426588,0.550940,0.004640,0.005482,0.004924,0.434149,0.005027,0.932087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.009933,0.000352,NaN,0.004142,0.001792,1.009866,0.003735,0.164614,0.948851,...,0.591469,0.029180,0.160292,0.360471,0.005373,0.004145,0.108002,0.830225,0.004286,0.932087
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,1.008022,1.005743,0.021199,0.007633,0.008006,0.055656,0.008819,0.164614,0.301055,...,0.042599,0.368335,0.563972,0.582941,0.004042,0.005473,0.086089,0.290629,0.003844,0.883089
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,1.002235,0.173585,NaN,0.007084,0.006079,1.007023,0.002009,0.164614,0.986411,...,0.268444,0.043031,0.350878,0.406980,0.005318,0.003546,0.033282,0.148788,0.004572,0.422141
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1.003983,0.509407,NaN,0.004469,0.000409,0.714486,0.004916,0.164614,0.902168,...,0.039741,0.018161,0.038328,0.713475,0.773797,0.004620,0.008030,0.213409,0.006008,0.375937


In [87]:
for dataset in ['train_data','train_data_emp1', 'test_data','test_data_emp1']:
    df = load_prepare_amex_dataset(dataset)
    nanmap = df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
    print(dataset, "nan", len([x for x in nanmap if x['nan_count']>0]))
    del df
gc.collect()

train_data nan 122
train_data_emp1 nan 51
test_data nan 121
test_data_emp1 nan 50


0

In [177]:
for dataset in ['train_data','train_data_emp1', 'test_data','test_data_emp1']:
    df = load_prepare_amex_dataset(dataset)
    nanmap = df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
    print(dataset, "nan", len([x for x in nanmap if x['nan_count']>0]))
    del df
gc.collect()

train_data nan 122
train_data_emp1 nan 35
test_data nan 121
test_data_emp1 nan 38


0

In [ ]:
%%time
# variant_df = variant_df[reversed(list(variant_df.columns))]
# we keep this copy so that original feature doens't stay on top!!!
# it's some weird quirk i found
# variant_df[missingvaluefeature+"_COPY"] = variant_df[missingvaluefeature]
# del variant_df[missingvaluefeature]
features_x = [f for f in df_train.columns if f != 'customer_ID' and f != 'target' and f!='S_2']
feature_y = 'target'
# df_x = df_train[features_x]
# df_y = df_train_wt[feature_y]
# target =

total_splits = 5
kf = StratifiedKFold(n_splits=total_splits, shuffle=True)
fold_scores = []
models = []
# NOT TOO SURE about feature_importances when all features are variants!! it's non decisive
for fold, (idx_train, idx_dev) in enumerate(kf.split(df_train, target)):
    print(f"------ FOLD: {fold+1} of {total_splits} -------")
    train_x, dev_x, train_y, dev_y, model = None, None, None, None, None
    start_time = datetime.now()

    X_tr = df_train.iloc[idx_train][features_x]
    X_va = df_train.iloc[idx_dev][features_x]
    y_tr = target[idx_train]
    y_va = target[idx_dev]
   
    model = create_lgbm_model_with_config(importance_type='gain')
    model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                  eval_metric=[lgbm_eval_metric_amex],
                  callbacks=[log_evaluation(100)])

    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = fast_amex_metric(y_va, y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print("-------------------------------------")
    display(HTML(f"<h3>Fold {fold+1} | {str(datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f} | Importance: Gain</h3>"))
    print("-------------------------------------")
    # fold_scores.append(score)
    track_score(features_x, score)
    models.append(model)
    # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importances_), key=lambda x:x[1])
    # # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importance(importance_type='gain')), key=lambda x:x[1])
    # feature_importance_map = {k:v for k,v in feature_importance_tuples}
    # feature_imp = pd.DataFrame(feature_importance_tuples, columns=['feature','importance'])
    # fig = pltex.bar(feature_imp, x='importance', y='feature', height=500)
    # fig.show()


    # track_score(features_x, score)
            # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importances_), key=lambda x:x[1])
            # # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importance(importance_type='gain')), key=lambda x:x[1])
            # feature_importance_map = {k:v for k,v in feature_importance_tuples}
            # feature_imp = pd.DataFrame(feature_importance_tuples, columns=['feature','importance'])
            # fig = pltex.bar(feature_imp, x='importance', y='feature', height=500)
            # fig.show()

show_score(features_x)
        
    # input("-------- ENTER TO CONINUE -------")
       

    

------ FOLD: 1 of 5 -------
[LightGBM] [Warning] num_threads is set=12, n_jobs=-1 will be ignored. Current value: num_threads=12
[LightGBM] [Warning] early_stopping_round is set=240, early_stopping_rounds=240 will be ignored. Current value: early_stopping_round=240


C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.229371	valid_0's amex: 0.789217
[200]	valid_0's binary_logloss: 0.217675	valid_0's amex: 0.79362
[300]	valid_0's binary_logloss: 0.215718	valid_0's amex: 0.796506
[400]	valid_0's binary_logloss: 0.214954	valid_0's amex: 0.798333
[500]	valid_0's binary_logloss: 0.214507	valid_0's amex: 0.798791
[600]	valid_0's binary_logloss: 0.214172	valid_0's amex: 0.79891
[700]	valid_0's binary_logloss: 0.213963	valid_0's amex: 0.799214
[800]	valid_0's binary_logloss: 0.213758	valid_0's amex: 0.798849
[900]	valid_0's binary_logloss: 0.213657	valid_0's amex: 0.799415
[1000]	valid_0's binary_logloss: 0.213672	valid_0's amex: 0.799898
[1100]	valid_0's binary_logloss: 0.213637	valid_0's amex: 0.799686
[1200]	valid_0's binary_logloss: 0.21361	valid_0's amex: 0.799702
[1300]	valid_0's binary_logloss: 0.213616	valid_0's amex: 0.800107
[1400]	valid_0's binary_logloss: 0.21364	valid_0's amex: 0.799435
-------------------------------------


-------------------------------------
------ FOLD: 2 of 5 -------
[LightGBM] [Warning] num_threads is set=12, n_jobs=-1 will be ignored. Current value: num_threads=12
[LightGBM] [Warning] early_stopping_round is set=240, early_stopping_rounds=240 will be ignored. Current value: early_stopping_round=240


C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

-------------------------------------
------ FOLD: 5 of 5 -------
[LightGBM] [Warning] num_threads is set=12, n_jobs=-1 will be ignored. Current value: num_threads=12
[LightGBM] [Warning] early_stopping_round is set=240, early_stopping_rounds=240 will be ignored. Current value: early_stopping_round=240


C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.23205	valid_0's amex: 0.782182
[200]	valid_0's binary_logloss: 0.221194	valid_0's amex: 0.785679
[300]	valid_0's binary_logloss: 0.219522	valid_0's amex: 0.787362
[400]	valid_0's binary_logloss: 0.218911	valid_0's amex: 0.78842
[500]	valid_0's binary_logloss: 0.218456	valid_0's amex: 0.789408
[600]	valid_0's binary_logloss: 0.218158	valid_0's amex: 0.789687
[700]	valid_0's binary_logloss: 0.218025	valid_0's amex: 0.789803
[800]	valid_0's binary_logloss: 0.217957	valid_0's amex: 0.789905
-------------------------------------


-------------------------------------


CPU times: total: 2h 57min 21s
Wall time: 15min 21s


In [85]:
gc.collect()

18763

In [23]:
%%time
# df_test = load_prepare_amex_dataset("test_data_emp1")
# df_test_last1 = df_test.groupby('customer_ID').last()
# df_test_last1 = df_test_last1.reset_index()
# df_test_wt = pd.merge(df_test_last1, train_label_df, how='inner', on = 'customer_ID')#.reset_index()

CPU times: total: 9min 3s
Wall time: 8min 2s


In [ ]:
df_test_bs_features = load_prepare_amex_dataset("test_data_bs_features_extracted")
df_test = df_test.reset_index().rename(columns={'index':'customer_ID'})
df_test = df_test.merge(df_test_bs_features, on="customer_ID", how="inner")

In [29]:
df_test

,customer_ID,customer_ID,D_54_last,B_16_last,D_49_last,S_20_last,R_5_last,B_2_last,R_14_last,D_138_last,...,bs_386,bs_387,bs_388,bs_389,bs_390,bs_391,bs_392,bs_393,bs_394,bs_395
0,0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,1.007686,0.002311,NaN,0.005336,0.005444,1.009347,0.006869,0.156909,...,-0.335811,-0.263116,-0.358178,0.651611,-0.356908,-0.478558,-0.367319,0.651469,0.656389,-0.642639
1,1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.003089,0.091107,NaN,0.002152,0.004132,1.009245,0.005801,0.156909,...,0.216133,-0.451675,-0.000535,0.589825,-1.719163,-0.615235,-0.863192,0.367502,0.924606,-0.155024
2,2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,1.003348,0.006028,NaN,0.004302,0.006620,0.810072,0.000496,0.156909,...,-0.100861,-0.195216,-0.379617,0.511827,-0.867911,-0.046278,-0.733860,0.115301,0.734151,-0.118117
3,3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,1.005929,1.006609,NaN,0.009516,0.002603,0.205678,0.002470,0.156909,...,0.031008,-0.076750,0.583770,0.474542,0.187113,0.799281,-0.935340,-0.552446,0.593164,0.096221
4,4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1.004245,1.009822,NaN,0.007866,0.004384,0.038021,0.007108,0.156909,...,0.083369,0.434476,0.689682,0.153366,-0.259030,0.776191,-0.317919,-0.485008,0.351477,0.877450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,1.002669,0.000557,NaN,0.004032,0.003819,0.817037,0.003745,0.156909,...,-0.019105,-0.194065,-0.032324,0.184423,-0.669451,-0.316483,-0.806006,0.021660,1.058970,-0.052874
924617,924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,1.005131,0.750046,NaN,1.009761,0.008875,0.082507,0.005900,0.156909,...,0.285859,0.001305,0.758463,-0.141299,-0.014544,0.542051,-0.877696,-0.496406,0.871348,0.833241
924618,924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,1.000603,0.003766,0.134571,0.007734,0.001188,1.004201,0.005715,0.156909,...,-0.385386,-0.395347,0.154520,0.268312,0.320566,0.802908,-0.973622,-0.388167,0.424626,0.147250
924619,924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,1.001342,1.003929,0.145777,0.006050,0.008189,0.192895,0.003014,0.156909,...,0.351461,-0.602158,0.656530,0.326861,0.595702,0.146379,-0.909322,-0.067012,0.954207,-0.149358


In [28]:
features_x = [f for f in df_test.columns if f not in ['customer_ID', 'target', 'prediction', 'S_2']]
X_tr = df_test[features_x]

predictions = []
for model in models:
    preds = model.predict_proba(X_tr)
    predictions.append(preds[:,1])#/len(models)
    


In [24]:
df_test

,customer_ID,D_105_last,B_41_last,B_6_last,D_47_last,S_11_last,S_19_last,B_17_last,D_71_last,D_62_last,...,D_43_avg,B_5_avg,B_28_avg,S_15_avg,B_32_avg,D_122_avg,D_91_avg,D_145_avg,B_3_avg,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.385235,0.006789,0.024945,0.489448,1.767255,0.003337,0.928880,0.281546,0.049326,...,0.006835,0.047263,0.145220,0.459957,0.005471,0.147003,2.340590,0.005271,0.009732,0.034984
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.385235,0.007340,0.182720,0.362383,0.128330,0.008257,0.928880,0.010397,0.330806,...,0.161138,0.101462,0.033780,0.106175,0.003917,0.576952,0.235882,0.005466,0.006160,0.001249
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.287646,0.007062,0.058534,0.204331,0.889807,0.005802,0.928880,0.044768,0.520753,...,0.130507,0.020789,0.147394,0.320956,0.004749,0.290148,0.003051,0.096248,0.008552,0.065729
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.224398,0.002427,0.023546,0.171934,0.968042,0.008854,0.668001,0.017329,0.016577,...,0.301156,0.031221,0.312598,0.274504,0.004543,0.147582,0.003694,0.186693,0.666754,0.292063
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.385235,0.004532,0.011244,0.039700,0.440953,0.001778,0.977441,0.009581,0.008594,...,0.226945,0.010636,0.089654,0.328378,0.004853,0.147947,0.004779,0.005032,0.616494,0.899910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.385235,0.007432,0.027490,0.686751,0.325048,0.005104,0.928880,0.223897,0.111100,...,0.044796,0.006056,0.020825,0.105243,0.003565,0.576284,0.004979,0.004533,0.024127,0.025200
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.227022,0.005822,0.038857,0.101321,0.281895,0.015289,0.799603,0.050171,0.025270,...,0.387672,0.024482,0.034786,0.512608,0.006330,0.290021,0.005509,0.005944,0.037561,0.842739
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.385235,0.007736,0.022962,0.334874,0.924145,0.001483,0.928880,0.024912,0.013268,...,0.170376,0.032785,0.160168,0.336011,0.004171,0.576455,0.004919,0.003697,0.028587,0.507823
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.143969,0.007967,0.008749,0.554004,0.406022,0.007858,0.492614,0.013216,0.076901,...,0.204906,0.012303,0.575861,0.474902,0.005058,0.190556,0.005707,0.004818,0.119271,0.247863


In [45]:
new_cols = []
for idx, x in enumerate(df_test.columns):
    if idx == 0:
        new_cols.append("asdf")
    else:
        new_cols.append(x)
        

In [47]:
df_test.columns = new_cols

In [49]:
df_test[['customer_ID','prediction']]

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.012867
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000692
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.038715
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.268484
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.822679
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.022296
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.827280
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.475655
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.299100


In [50]:
df_test['prediction'] = np.mean(predictions, axis=0)
df_test[['customer_ID','prediction']].to_csv("submission.csv",index=False)

In [51]:
pd.read_csv("submission.csv")

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.009875
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000525
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.032084
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.237248
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.804010
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.016307
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.781760
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.435008
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.237427


```
Data: EMP1, + NN Sequential NN
Features: similar aggregation like https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart, except Cat aggs
FEATURES: BS MOde feature extractors
Model: 5 fold, lgbm standard, based on Sequence Features extracted
Ensemble: choose max prediction value;highest proba out of 5 models
```